In [16]:
from sklearn.ensemble import RandomForestRegressor
## Install XGBOOST in the Anaconda envo using:  conda install -c conda-forge xgboost 
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math
from statistics import mean
import seaborn as sns 
%matplotlib inline 
import xgboost as xgb
from xgboost import XGBClassifier
from scipy.stats import pearsonr
from sklearn import tree, linear_model
from sklearn.metrics import explained_variance_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from timeit import default_timer
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix


In [17]:
# Read the data into a data frame
supData = pd.read_csv("train.csv", delimiter=",")
print("length of supData: ", len(supData))
# Check the number of features in the data set
print("features in the data set: ", len(supData.columns))
# Check the data types
print("data type: ", supData.dtypes.unique())

# Check any number of columns with NaN
print("Check NaN in coloumn : ", supData.isnull().any().sum(), ' / ', len(supData.columns))
# Check any number of data points with NaN
print("Check NaN in rows : ", supData.isnull().any(axis=1).sum(), ' / ', len(supData))


# Read the unique_m.csv data
supData_ele = pd.read_csv("unique_m.csv", delimiter=",")
supData_ele_ = supData_ele[['Cu','O','Fe','material']] # select columns contain Cu, O, Fe

bigdata =pd.concat([supData, supData_ele_], axis=1) # To merge across columns
print("length of bigdata: ", len(bigdata))
#bigdata.to_csv('file_name.csv', sep='\t') # To save the output data to a .csv file

supData_ele_Cu = bigdata.loc[~bigdata['Cu'].isin(['0'])] # Delete 0 for the Cu column
supData_ele_O = bigdata.loc[~bigdata['O'].isin(['0'])] # Delete 0 for the O column
supData_ele_Fe = bigdata.loc[~bigdata['Fe'].isin(['0'])] # Delete 0 for the Fe column




print("length Cu: ", len(supData_ele_Cu), "length of O: ", len(supData_ele_O), "length of Fe", len(supData_ele_Fe))
#supData_ele_Cu.to_csv('file_name_Cu.csv', sep='\t') # To save the output data to a .csv file
#supData_ele_O.to_csv('file_name_O.csv', sep='\t') # To save the output data to a .csv file
#supData_ele_Fe.to_csv('file_name_Fe.csv', sep='\t') # To save the output data to a .csv file

length of supData:  21263
features in the data set:  82
data type:  [dtype('int64') dtype('float64')]
Check NaN in coloumn :  0  /  82
Check NaN in rows :  0  /  21263
length of bigdata:  21263
length Cu:  10838 length of O:  11964 length of Fe 2339


In [18]:
def load_Data(Data):
    myData = pd.DataFrame(Data)
    #--------------
    # Extract features and target data
    target = myData.iloc[:,-5] # get the 5th column from the last of the data frame
    features = myData.drop(['critical_temp','Cu', 'O', 'Fe','material'], axis=1)
    features_normalized = preprocessing.normalize(features, norm='l2', axis=1, copy=True, return_norm=False)

    X_train, X_test, y_train, y_test = train_test_split(features_normalized, target, test_size=0.33)
    #return X_train, X_test, y_train, y_test, features, target #return already normalized features.
    
    return X_train, X_test, y_train, y_test


def Grid_Search_CV_SVM(X_train, y_train):
    estimator = SVR()

    parameters = {'kernel': ['sigmoid', 'rbf'], 
              'gamma': ['auto'], 
               'coef0': [0.0, 0.1],
             }

    grid = GridSearchCV(estimator, parameters, n_jobs=-1, cv=10)

    grid.fit(X_train, y_train)

    return grid.best_score_ , grid.best_params_

def SVM(X_train, X_test, y_train, y_test, best_params):
    estimator = SVR().set_params(**best_params)
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
    return y_test,y_predict

In [19]:
## O based superconductors

for i in range(3):
    print ("Loop: " , i)
    print ("--------------")
    X_train, X_test, y_train, y_test = load_Data(supData_ele_O)
    best_score, best_params = Grid_Search_CV_SVM(X_train, y_train)
    y_test , y_predict = SVM(X_train, X_test, y_train, y_test, best_params)

    print ("Best Score:" ,best_score)
    print ("Best params:",best_params)

Loop:  0
--------------
Best Score: 0.0010667623111927714
Best params: {'coef0': 0.0, 'gamma': 'auto', 'kernel': 'rbf'}
Loop:  1
--------------
Best Score: 0.0001142351592256207
Best params: {'coef0': 0.0, 'gamma': 'auto', 'kernel': 'rbf'}
Loop:  2
--------------
Best Score: -0.0005175159468432604
Best params: {'coef0': 0.0, 'gamma': 'auto', 'kernel': 'rbf'}


In [15]:
## Copper based superconductors
for i in range(3):
    print ("Loop: " , i)
    print ("--------------")
    X_train, X_test, y_train, y_test = load_Data(supData_ele_Cu)
    best_score, best_params = Grid_Search_CV_SVM(X_train, y_train)
    y_test , y_predict = SVM(X_train, X_test, y_train, y_test, best_params)

    print ("Best Score:" ,best_score)
    print ("Best params:",best_params)

Loop:  0
--------------
Best Score: -0.0035257088952374413
Best params: {'coef0': 0.0, 'gamma': 'auto', 'kernel': 'rbf'}
Loop:  1
--------------
Best Score: 0.000437690977101574
Best params: {'coef0': 0.0, 'gamma': 'auto', 'kernel': 'rbf'}
Loop:  2
--------------
Best Score: -0.0017181939292441068
Best params: {'coef0': 0.0, 'gamma': 'auto', 'kernel': 'rbf'}


In [1]:
## SVM performs very poorly for this dataset